<a href="https://colab.research.google.com/github/kentokura/gob/blob/main/gogogobb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythonでゴブレットゴブラーズのゲーム木を作成する

- [ゴブレットゴブラーズ (日本語版)](https://sgrk.blog.fc2.com/blog-entry-3687.html)
- [ソースコード設計スライド](https://docs.google.com/presentation/d/1K8fbRlk24Y_J3M--F-WnYYN0hddL89w3EGN4Zgw-L1g/edit?usp=sharing)

## stateを定義

set_stateとchoise_stateの２種類がある。  
必要な情報を84bitで表す。  
state = 0b_0_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_000000000000


| 区分 | is_first | is_choise | is_choise_board | choise_piece_type | choise_piece | board | hand |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| bit数 | 1bit | 1bit | 1bit | 6bit | 9bit | 54bit | 12bit |
| 例 | 1 | 1 | 1 | 100000 | 100000000 | 010000000 001000000 000110000 000000000 000000000 000000000 | 00 00 00 00 00 00 |
| 説明 | 手番 | モード | 持ち上げている駒は盤か手か | 持ち上げている駒の種類[rl,rm,rs,bl,bm,bs] | 持ち上げている駒のマス[0,1,2,3,4,5,6,7,8] | 盤面に置いてある駒[rl0, rl1, ..., rm0..., bl0...] | 手駒[rlの個数, rmの...] |

説明の補足 :   
- hand  “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる


In [2]:
# 確認
state = 0b_1_0_0_100000_100000000_100000000000000000000000000000000000000000000000000000_100000000000
print(state) # 10進数
print(f'{state:084b}')#bin(プレフィックスなし)
print(type(f'{state:084b}')) #文字列です
print(len(f'{state:084b}')) # bit数の確認
# 1bit取り出す
print(f'{state:084b}'[0])
# 複数のbitを取り出す
print(f'{state:084b}'[1+1+1:1+1+1+6])
# 1bit書き換え
print("変更前 : " + f'{state:084b}')
state = int(f'{state:084b}'[:1] + '1' + f'{state:084b}'[2:], 2)
print("変更後 : " + f'{state:084b}')
# 複数のbitを書き換え
print("変更前 : " + f'{state:084b}')
state = int(f'{state:084b}'[:2] + '1' + f'{state:084b}'[3:72] + '111111000000', 2)
print("変更後 : " + f'{state:084b}')

# バラす
is_first = f'{state:084b}'[0]
is_choise = f'{state:084b}'[1]
is_choise_board = f'{state:084b}'[2]
choise_piece_type = f'{state:084b}'[3:3+6]
choise_piece = f'{state:084b}'[9:9+9]
board = f'{state:084b}'[18:18+54]
hand = f'{state:084b}'[72:72+12]
print("バラす  : " + is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand)

10899258735951288572315648
100100000100000000100000000000000000000000000000000000000000000000000000100000000000
<class 'str'>
84
1
100000
変更前 : 100100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更後 : 110100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更前 : 110100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更後 : 111100000100000000100000000000000000000000000000000000000000000000000000111111000000
バラす  : 111100000100000000100000000000000000000000000000000000000000000000000000111111000000


## actionを定義

set_action, choise_actionともに、行動する駒を表す。

必要な情報を16bitで表す。  
action = 0b_0_000000_000000000


| 区分 | is_choise_board | choise_piece_type | choise_place |
| ---- | ---- | ---- | ---- | 
| bit数 | 1bit | 6bit | 9bit | 
| 例 | 1 | 100000 | 100000000 | 
| 説明 | 行動する駒は盤か手か | 行動する駒の種類[rl,rm,rs,bl,bm,bs] | 行動する駒のマス[0,1,2,3,4,5,6,7,8] | 


In [3]:
# 確認
action = 0b_1_100000_100000000
print(action) # 10進数
print(f'{action:016b}')#bin(プレフィックスなし)
print(type(f'{action:016b}')) #文字列です
print(len(f'{action:016b}')) # bit数の確認
# 1bit取り出す
print(f'{action:016b}'[0])
# 複数のbitを取り出す
print(f'{action:016b}'[1:1+6])
# 書き換え
print("変更前 : " + f'{action:016b}')
action = int(f'{action:016b}'[:1] + '000001' + '000000010', 2)
print("変更後 : " + f'{action:016b}')


# バラす
is_choise_board = f'{action:016b}'[0]
choise_piece_type = f'{action:016b}'[1:1+6]
choise_piece = f'{action:016b}'[7:7+9]
print("バラす  : " + is_choise_board + choise_piece_type + choise_piece)

49408
1100000100000000
<class 'str'>
16
1
100000
変更前 : 1100000100000000
変更後 : 1000001000000010
バラす  : 1000001000000010


## stateからその合法手一覧であるactionsを作成する関数

合法手をすべて列挙する... actions = [action1, action2, …]  
leagal_actions(state) -> actions: を作成
```
def leagal_actions(state) -> actions:
  actions = []
  もし stateがchoise_stateなら、
    actions = leagal_actions_from_choise_state(choise_state)
  そうでなく stateがset_stateなら、
    actions = leagal_actions_from_set_state(set_state)
  return actions
```
stateごとに内部関数を作成。  
- choise_stateから合法手を作成 : leagal_actions_from_choise_state(choise_state)
- set_stateから合法手を作成 : leagal_actions_from_set_state(set_state)

### choise_stateから合法手を作成

  
leagal_actions_from_choise_state(choise_state) -> set_actions:  
setする駒と場所一覧... set_actions を作成する  
setできる駒と場所とは....
- handは行動できない。あくまで選んだ駒を盤面に配置するという作業だから。
- choiseしている駒と同じ駒しか行動できない。例: rlを選んだならrlしか行動できない。
- choiseした駒以上の大きさの駒があるマスには行動できない。
- choiseした駒が盤面なら、
    - choiseした駒があったマスには行動できない。

In [97]:
def leagal_actions_from_choise_state(choise_state):
  is_choise_board = f'{choise_state:084b}'[2]
  choise_piece_type = f'{choise_state:084b}'[3:3+6]
  choise_piece = f'{choise_state:084b}'[9:9+9]
  board = f'{choise_state:084b}'[18:18+54]

  # set_actions_bitを作成
  set_actions_bit = f'{0:054b}' # 54bit。すべての駒と位置が行動不能で初期化
  # choiseしている駒と同じ駒は行動できる。
  cpy_f = choise_piece_type.find('1')
  set_actions_bit = set_actions_bit[:9*cpy_f] + '111111111' + set_actions_bit[9*(cpy_f+1):]
  # choiseしている駒がboardの駒なら、そのマスに行動出来ない
  if is_choise_board == '1': 
    cp_f = choise_piece.find('1')
    set_actions_bit = set_actions_bit[:9*cpy_f+cp_f] + '0' + set_actions_bit[9*cpy_f+cp_f+1:]
  # choiseした駒以上の大きさの駒があるマスには行動できない。
  for mass in range(9):
    if board[mass] == '1' or board[mass+27] == "1": # L駒があるか
      set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [1, 2, 4, 5]: #選択してあるこまがM, Sなら
      if board[mass+9] == '1' or board[mass+27+9] == "1": # M駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [2, 5]: #選択してあるこまがSなら
      if board[mass+18] == '1' or board[mass+27+18] == "1": # S駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]

  # actions_bitをactionsに変換
  set_actions = []
  for i, bit in enumerate(set_actions_bit):
    if bit == '1':
      set_action = int('0b_1_000000_000000000', 2)
      set_action = int(f'{set_action:016b}'[:1+(i//9)] + '1' + f'{set_action:016b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      set_action = int(f'{set_action:016b}'[:1+6+(i%9)] + '1' + f'{set_action:016b}'[1+6+(i%9)+1:], 2) # 駒の位置の反映
      set_actions.append(set_action)

  return set_actions

In [113]:
# 確認

# choiseしている駒と同じ駒は行動できる。

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_100000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_1_1_0_010000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_110111111111
print("先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ
state = 0b_0_1_0_000001_000000000_000000000_000000000_000000000_000000000_000000000_000000000_111111111101
print("後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
print()

# choiseしている駒がboardの駒なら、そのマスに行動出来ない

# 先手, マス0の自駒Lを選択時, 行動位置は1~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_100000_100000000_000000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は0~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, マス3の自駒Sを選択時, 行動位置は0~2, 4~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_001000_000100000_000000000_000000000_000000000_000000000_000000000_000000000_111101111111
print("先手, マス3の自駒Sを選択時, 置ける駒はS, その位置は0~2, 4~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
print()

# choiseした駒以上の大きさの駒があるマスには行動できない。

# 先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~8のハズ
state = 0b_1_1_0_100000_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ
state = 0b_1_1_0_010000_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 後手, 手駒Sを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2のハズ
state = 0b_1_1_0_000001_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("後手, 手駒Sを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
1100000100000000
1100000010000000
1100000001000000
1100000000100000
1100000000010000
1100000000001000
1100000000000100
1100000000000010
1100000000000001
先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
1010000100000000
1010000010000000
1010000001000000
1010000000100000
1010000000010000
1010000000001000
1010000000000100
1010000000000010
1010000000000001
後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ
1000001100000000
1000001010000000
1000001001000000
1000001000100000
1000001000010000
1000001000001000
1000001000000100
1000001000000010
1000001000000001

先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は0~8のハズ(盤面に他の駒なし)
1100000010000000
1100000001000000
1100000000100000
1100000000010000
1100000000001000
1100000000000100
1100000000000010
1100000000000001
先手, マス3の自駒Sを選択時, 置ける駒はS, その位置は0~2, 4~8のハズ(盤面に他の駒なし)
1001000100000000
1001000010000000
1001000001000000
1001000000010000
1001000000001000
1001000000000100
1001000000000010
1001000000000001

先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~

### set_stateから合法手を作成

leagal_actions_from_set_state(set_state) -> choise_actions:  
choiseできる駒の一覧... choise_actions を作成する  
choiseできる駒とは....  
- 自分の駒しか行動できない

手駒なら,
- 手駒に動かしたい駒が１つもなければ、その駒は行動できない。
- 動かしたい駒以上の大きさの駒が配置されているマスには行動できない。

盤上の駒なら、
- 盤上の一番表面にある駒しか行動できない。  ※後述するsurface_boardを使用
- 動かしたい駒以上の大きさの駒があるマスには行動できない。

### 合法手をすべて列挙する

In [120]:
def legal_actions(state):
  actions = []
  print(f'{state:b}'.ljust(84, '0'))
  # print("is_choise is {}".format(int(bin(state)[2])))
  if int(f'{state:b}'.ljust(84, '0')[1]) == '1':
    print("if")
    # actions = leagal_actions_from_choise_state(choise_state)
    pass;
  else:
    # actions = leagal_actions_from_set_state(set_state)
    pass;
  return actions

In [124]:
# 確認
state = 0b_0_1_0_000000_000000000_000000000000000000000000000000000000000000000000000000_000000000000
print(f'{state:b}'.ljust(84, '0'))
legal_actions(state)

100000000000000000000000000000000000000000000000000000000000000000000000000000000000
100000000000000000000000000000000000000000000000000000000000000000000000000000000000


[]